In [2]:
import numpy as np
import random
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_mat import *
from ids import *
from block_maxvol import *
from block_rect_maxvol import *
import scipy.linalg
from gen_1D import matrix_prep
import os
%matplotlib inline

In [4]:
nder = 2 # dimension
n = 2 #nmber of sampling points 
n_test = 5000 #points on test grid
p_size = (nder+1)*2 #number of monoms (number of columns)
interval = [-2, 2]  # interval of x_test
#np.random.seed(0)
x = 2*np.random.rand(n, nder) - 1

In [ ]:
c = GenMat(p_size, x,poly=cheb, debug=False,pow_p=1)
print c.shape
# turn matrix to the block structure
A = matrix_prep(c,nder+1,n)
print A.shape

In [5]:
#### PLUQ TESTING
row_exp_max = 10
col_exp_max = 5
nder = 2
ndim = nder+1
n_iter = int(1e5)
# treshold = 1e-05 # We do not use threshold
dir_str = '.'
saves = dict()
dir_matrix = os.path.join(dir_str, "matrices1")
try:
    os.mkdir(dir_matrix)
except:
    pass

si_count = 0

for i in range(n_iter):
    col_exp = np.random.randint(1,col_exp_max)
    row_exp = np.random.randint(col_exp,row_exp_max)
    A = np.random.rand(row_exp*ndim,col_exp*ndim)
    try:
        p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
        u = np.triu(lu[:A.shape[1]])
        s = la.svd(u,compute_uv=False)

        cur_thre = min(s[-1], np.abs(u[-1,-1]))
        try:
            to_save = saves[(row_exp, col_exp)] > cur_thre
        except:
            to_save = True

        if to_save:
        #if (s[-1] < treshold) or (np.abs(u[-1,-1]) < treshold):
            np.savez(os.path.join(dir_matrix,"pluq_row_exp={}_col_exp={}".format(row_exp, col_exp)), A = A)
            saves[(row_exp, col_exp)] = cur_thre

    except SingularError:
        np.savez(os.path.join(dir_matrix,"singular_row_exp={}_col_exp={}_i={}".format(row_exp, col_exp, si_count)), 
                 A = A)
        si_count += 1

### Processing of obtained matrices

In [ ]:
for i in range(1,10):
    for j in range(1,6):
        ### Try to open file with any combination of indices
        try:
            data = np.load('./matrices/pluq_row_exp={}_col_exp={}.npz'.format(i,j))
            A = data['A']
            Fl = True
        except:
            ### File does not exist, just skip part that is below
            Fl = False
            pass  
        if Fl == True:
            try:
                p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
                perm = A[p][:,q]
                s_A = la.svd(perm[:A.shape[1]],compute_uv=False)
                u = np.triu(lu[:A.shape[1]])
                s = la.svd(u,compute_uv=False)
                print (s_A[-1], s[-1], u[-1,-1], la.det(u), la.det(perm[:A.shape[1]]))
            except SingularError as err:
                print 'not full column rank', err.value

In [9]:
### SINGLE EXAMPLE
data = np.load('./matrices/pluq_row_exp=5_col_exp=3.npz')
A = data['A']
p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
perm = A[p][:,q]
s_A = la.svd(perm[:A.shape[1]],compute_uv=False)
u = np.triu(lu[:A.shape[1]])
s = la.svd(u,compute_uv=False)
print ('Singular values of U = {}'.format(s))
print ('Singular values of quadr PAQ = {}'.format(s_A))
print ('det of U = {}, det of quadr PAQ = {}'.format(la.det(u), la.det(perm[:A.shape[1]])))

Singular values of U = [ 1.98553705  1.92155774  1.43717761  1.06675098  0.81780971  0.61118696
  0.52191605  0.3296006   0.16127143]
Singular values of quadr PAQ = [ 5.04323496  1.66722575  1.32789736  0.87368986  0.83780067  0.73293508
  0.49004275  0.31838457  0.08678801]
det of U = -0.0811105497923, det of quadr PAQ = -0.0811105497923


# RGV 2
### Test for linear dependency. 1st thought about pullbacks

In [ ]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

In [ ]:
np.linalg.matrix_rank(mat_test)

## Check, that matrix is good

In [ ]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

In [ ]:
p,l,u,q, info = pluq_ids(mat_test, debug=True)
print info

# Another Error 
### That was about columns swap. 
If piv[0] == j, piv[1] != j+1

In [ ]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids_pull(mat_test_5, debug=True)

In [ ]:
p2,l2,u2,q2,inf = pluq_ids(mat_test_5, debug = True)

In [ ]:
print(la.det(np.dot(perm_matrix(p),mat_test_5)[:6]))
print(la.det(np.dot(perm_matrix(p2),mat_test_5)[:6]))
print p,p2

## Yet another bug
About feature of maxvol's performance. It returns error if we have a long pair consisted from more then 2 elements each with rank < 2

In [ ]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_6, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_6)

In [ ]:
print(la.det(np.dot(perm_matrix(p),mat_test_6)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_6)[:6]))
print p,p1

## Yet another bug ver. 2.0: Division by Zero
### This and below: again about columns swap. Solved

In [ ]:
mat_test_7 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_7, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_7)

In [ ]:
print(la.det(np.dot(perm_matrix(p),mat_test_7)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_7)[:6]))
print p,p1
print q,q1

## Yet another bug ver. 3.0: Division by Zero

In [ ]:
mat_test_8 = np.array([[ -5,  -9, -13,  -2,  19,  11],
       [ 14, -17,  -8,   2, -12,   0],
       [ 17, -19,  -3,   6, -19,   1],
       [ 19,   0, -16, -19,  19,  18],
       [ -5,   2,   1,   4, -17,   1],
       [  4,   2,   7,   9, -10, -15],
       [ 19,  -9,   0, -12,   0,  -6],
       [-10,  15,  12,   2,   2,  13],
       [ 16,   0,   3, -16,  16, -15],
       [-15, -14,   9,   8,   0,  17]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_8, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_8)

In [ ]:
print(la.det(np.dot(perm_matrix(p),mat_test_8)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_8)[:6]))
print p,p1

In [ ]:
hap = p_preproc(p,2)
print hap

In [ ]:
print(la.det(np.dot(perm_matrix(hap),mat_test_8)[:6]))

In [ ]:
indx_n, indx_o = change_intersept(np.arange(4),(np.arange(4))[[1,3,0,2]])
#print indx_n,indx_o
#aab = np.concatenate((indx_n,indx_o))
#bba = np.concatenate((indx_o,indx_n))
#print aab,bba

mat_test_8[indx_n,:] = mat_test_8[indx_o,:]


In [ ]:
mat_test_8